In [6]:
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
from datetime import datetime as dt
import re
import time
from IPython.display import clear_output
import os

In [9]:
def enlace_to_rowdf(enlace):
    new_row = dict()
    new_row['Código inmueble'] = 'UNASIG 000'
    new_row['Enlace'] = enlace
    obs_text = ''
    try:
        page = urlopen(enlace)
        html_bytes = page.read()
        html = html_bytes.decode("utf-8")
        soup = BeautifulSoup(html, 'html.parser')

        try:
            data_raw_1 = soup.find('title')
            new_row['Tipo'] = data_raw_1.text.split(' ')[2]
        except:
            obs_text = obs_text + 'sin info_2'

        try:
            data_raw_2 = soup.find('p', attrs={"class": "P-sc-31hrrf-0 hGwghD card-subtitle"})
            new_row['Sector'] = ', '.join(data_raw_2.text.split(',')[0].upper().replace('SECTOR ', '').replace(' Y ALREDEDORES', '').split(' Y '))
        except:
            obs_text = obs_text + 'sin info_2'

        try:
            data_raw_3 = soup.find('ul', attrs={"class": "Ul-sctud2-0 kdIYsW inline-list-grid list-feature-detail"})
            data_raw_3 = data_raw_3.find_all('li')
            new_row['Habitaciones'] = int(data_raw_3[1].text[0])
            new_row['Baños'] = int(data_raw_3[2].text[0])
            new_row['Estrato'] = int(data_raw_3[3].text[0])
        except:
            new_row['Habitaciones'] = np.NaN
            new_row['Baños'] = np.NaN
            new_row['Estrato'] = np.NaN
            obs_text = obs_text + ', '+ 'sin info_3'

        try:
            data_raw_4 = soup.find_all(attrs={"class": "Col-sc-14ninbu-0 lfGZKA mb-3 pb-1 col-12 col-lg-3"})
            for element in data_raw_4:
                if element.h3.text == 'Precio' or element.h3.text == 'Valor administración':
                    text = int(element.p.text[1:].replace('.', ''))
                elif element.h3.text == 'Área construida' or element.h3.text == 'Área privada':
                    text = float(element.p.text[:-2].replace(' ', ''))
                elif element.h3.text == 'Parqueaderos':
                    text = int(element.p.text)
                elif element.h3.text == 'Barrio común':
                    text = element.p.text.upper()
                else:
                    text = element.p.text
                new_row[element.h3.text] = text
        except:
            obs_text = obs_text + ', '+  'sin info_4'
            new_row['Observaciones'] = obs_text

        try:
            card_block = soup.find_all('div', attrs={"class": "card-block"})
            try:
                lista_interiores_raw = card_block[3].find_all('ul', attrs={"class": "Ul-sctud2-0 kdIYsW inline-list-grid"})
                lista_interiores = list()
                for item in lista_interiores_raw:
                    lista_interiores.append(item.text)
                new_row['Interiores'] = ', '.join(lista_interiores)
            except:
                obs_text = obs_text + ', '+  'sin info_5.1'
                new_row['Observaciones'] = obs_text

            try:
                lista_exteriores_raw = card_block[4].find_all('ul', attrs={"class": "Ul-sctud2-0 kdIYsW inline-list-grid"})
                lista_exteriores = list()
                for item in lista_exteriores_raw:
                    lista_exteriores.append(item.text)
                new_row['Exteriores'] = ', '.join(lista_exteriores)
            except:
                obs_text = obs_text + ', '+  'sin info_5.2'
                new_row['Observaciones'] = obs_text
            try:
                lista_zonascomunes_raw = card_block[5].find_all('ul', attrs={"class": "Ul-sctud2-0 kdIYsW inline-list-grid"})
                lista_zonascomunes = list()
                for item in lista_zonascomunes_raw:
                    lista_zonascomunes.append(item.text)
                new_row['Zonas comunes'] = ', '.join(lista_zonascomunes)
            except:
                obs_text = obs_text + ', '+  'sin info_5.3'
                new_row['Observaciones'] = obs_text
        except:
            obs_text = obs_text + ', '+  'sin info_5'
            new_row['Observaciones'] = obs_text

        try:
            box_2 = soup.find('script', attrs={"id": "__NEXT_DATA__"})
            try:
                # Ubbicación / posición de coordenadas
                new_row['Ubicacion'] = json.loads(box_2.get_text())['props']['initialState']['realestate']['basic']['coordinates']
            except:
                new_row['Ubicacion'] = {'lon': None, 'lat': None}
                obs_text = obs_text + ', '+  'ubicación no disponible'
                new_row['Observaciones'] = obs_text
            try:
                # Imagenes
                list_images = json.loads(box_2.get_text())['props']['initialState']['realestate']['basic']['images']
                new_list_img = list()
                for img in list_images:
                    new_list_img.append(img['image'])
                new_list_img
                new_row['lista_imagenes'] = new_list_img
            except:
                new_row['lista_imagenes'] = 'NoInfo'
                obs_text = obs_text + ', '+  'imágenes no disponible'
                new_row['Observaciones'] = obs_text
            lat = new_row['Ubicacion']['lat']
            lon = new_row['Ubicacion']['lon']
            if lat is not None:
                new_row['G_maps'] = f'https://www.google.com/maps/place/{lat},{lon}'
            else:
                new_row['G_maps'] = 'NoInfo'
        except:
            pass

        try:
            box_3 = soup.find('p', attrs={"class": "d-none d-md-block card-text"})
            new_row['Descripcion'] = box_3.text
        except:
            new_row['Descripcion'] = 'NoInfo'


    except:
        obs_text = 'Sin acceso a enlace'
    new_row['Observaciones'] = obs_text
    return new_row

In [10]:
enlace = 'https://www.metrocuadrado.com/inmueble/venta-apartamento-bogota-caobos-salazar-3-habitaciones-2-banos-2-garajes/2974-M3237376'
enlace_to_rowdf(enlace)

{'Código inmueble': '2974-M3237376',
 'Enlace': 'https://www.metrocuadrado.com/inmueble/venta-apartamento-bogota-caobos-salazar-3-habitaciones-2-banos-2-garajes/2974-M3237376',
 'Tipo': 'Apartamento',
 'Sector': 'CEDRITOS',
 'Habitaciones': 3,
 'Baños': 2,
 'Estrato': 4,
 'Barrio común': 'CAOBOS SALAZAR',
 'Precio': 430000000,
 'Antigüedad': 'Entre 10 y 20 años',
 'Área construida': 100.0,
 'Área privada': 100.0,
 'Valor administración': 333500,
 'Parqueaderos': 2,
 'Interiores': 'Citófonos, Estudio o biblioteca, Parqueadero cubierto, Zona de lavandería, Instalación de gas natural, Piso 3, Calentador gas, Tipo de cocina abierta, Tipo de comedor sala comedor, Deposito 1, Tipo de estufa gas, Tipo de parqueadero propio, Tipo acabado de piso laminado, Vista interior',
 'Exteriores': 'Ascensor, Circuito cerrado de TV, Conjunto cerrado, Número de Ascensores 1, Vigilancia 24hrs',
 'Zonas comunes': 'Gimnasio, Salón comunal',
 'Ubicacion': {'lon': -74.032524, 'lat': 4.7297664},
 'lista_imagenes